In [2]:
import requests
import json
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


page_num = 1
last_page = False
total_results = []

# https://paulas-choice-singapore.myshopify.com/admin
# https://{username}:{password}@{shop}.myshopify.com/admin/api/{api_version}/{resource}.json

username = '7e86d45b24d9a3da5da72a93a8210f34'
password = '3ddc548cff03ffd7d064c7e2fa7f3431'
shop = 'paulas-choice-singapore'
api_version = '2020-04'
resource = 'orders'

while last_page == False:
    url = f'https://{username}:{password}@{shop}.myshopify.com/admin/api/{api_version}/{resource}.json?limit=20'
#     print(f"Printing page {url}")
    result = requests.get(url)
    data = result.json()
    total_results = total_results + data['orders']
    last_page = json.loads(result.headers['X-Pagination'])['last_page']
    page_num += 1
    print(f"last_page has value {last_page}")


KeyError: 'composition'

In [9]:
pd.Series(result.headers)

Date                                                      Wed, 10 Jun 2020 05:53:43 GMT
Content-Type                                            application/json; charset=utf-8
Transfer-Encoding                                                               chunked
Connection                                                                   keep-alive
Set-Cookie                            __cfduid=d5d0534aea6e6d3e1e36423fa3cea705f1591...
X-Sorting-Hat-PodId                                                                  64
X-Sorting-Hat-ShopId                                                           14402440
Vary                                                                    Accept-Encoding
Referrer-Policy                                                origin-when-cross-origin
X-Frame-Options                                                                    DENY
X-ShopId                                                                       14402440
X-ShardId                       

In [14]:
result.json()['orders']

[{'id': 2266326204481,
  'email': 'chris.wanling@gmail.com',
  'closed_at': None,
  'created_at': '2020-06-10T13:48:35+08:00',
  'updated_at': '2020-06-10T13:48:40+08:00',
  'number': 52648,
  'note': None,
  'token': 'e07f0dc13a1406a2b2d72d8c50d53fe8',
  'gateway': 'stripe',
  'test': False,
  'total_price': '45.00',
  'subtotal_price': '45.00',
  'total_weight': 158,
  'total_tax': '0.00',
  'taxes_included': False,
  'currency': 'SGD',
  'financial_status': 'paid',
  'confirmed': True,
  'total_discounts': '0.00',
  'total_line_items_price': '45.00',
  'cart_token': 'e77f4cceef45ec0bf9def53f02c710d6',
  'buyer_accepts_marketing': False,
  'name': '191569953648',
  'referring_site': '',
  'landing_site': '/',
  'cancelled_at': None,
  'cancel_reason': None,
  'total_price_usd': '32.43',
  'checkout_token': '204fc2ce162af5cea4becff13efc1079',
  'reference': None,
  'user_id': None,
  'location_id': None,
  'source_identifier': None,
  'source_url': None,
  'processed_at': '2020-06-10T

## BQ Insert Script

In [ ]:
import json
import requests
from google.cloud import bigquery

client = bigquery.Client()

def trade_gecko_to_bq(request):
    params = request.get_json(force=True)
    
    if not 'project' in params or not 'dataset' in params or not 'table' in params:
        return "Missing project, BigQuery dataset or table argument!"
        
    bq_table = params['project'] + '.' + params['dataset'] + '.' + params['table']
    
    try:
        client.get_table(bq_table)
    except:
        # Most likely table is non-existent
        return "Project, BigQuery dataset or table non-existent!"    
    
    url = 'https://api.tradegecko.com/variants'
    token = 'f4855aebc4a92c0d6a09f07b105bcbae81afbaf8cb1344f47a5b5c45cf8f4c1e'
    result = get_and_insert(bq_table, url, token)
    
    return json.dumps(result)

        
def get_and_insert(bq_table, url, token):
    
    bearer = {'Authorization': f'Bearer {token}'}
    limit_num = 250
    page_num = 1
 
    total_records = 0
    fail_records_to_insert = 0
    result = { 'status': 'success' }
    page_loop = True
    while page_loop:
        url = f'{url}?limit={limit_num}&page={page_num}'
        resp = requests.get(url, headers=bearer)
        if resp.status_code == 200:
            json_data = resp.json()
            
            if 'variants' in json_data:
                insert_responses = insert_to_bq(bq_table, json_data['variants'])
                
                if 'inserts' not in result:
                    result['inserts'] = []
                result['inserts'].append({ 'page': page_num, 'insert_responses': insert_responses })
                
                pagination = json.loads(resp.headers['X-Pagination'])
                total_records = pagination['total_records']
                page_loop = not pagination['last_page']
                page_num += 1
                fail_records_to_insert += len(insert_responses)
            else:
                result['status'] = 'failure'
                result['message'] = 'No "variants" attribute in text returned by Trade Gecko HTTP response'
                return result
        else:
            result['status'] = 'failure'
            result['message'] = 'Trade Gecko HTTP Response not 200 status code'
            return result
    
    result['total_records'] = f'{total_records - fail_records_to_insert} of {total_records} successfully inserted to BigQuery table `{bq_table}`'
    return result
                
                
def insert_to_bq(bq_table, data):
    for variant in data:
        variant['upc'] = None if variant['upc'] == '' else variant['upc']
        variant['stock_levels'] = convert_to_list(variant['stock_levels'], 'stock_id', 'level')
        variant['committed_stock_levels'] = convert_to_list(variant['committed_stock_levels'], 'stock_id', 'level')
        variant['prices'] = convert_to_list(variant['prices'], 'type', 'value')

    return client.insert_rows_json(table=bq_table, json_rows=data)
    

def convert_to_list(json_dict, key_label, value_label):
    if len(json_dict) == 0:
        return []
    else:
        return [{key_label: k, value_label: json_dict[k]} for k in json_dict]
